<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [6]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


And then we import our movie database

In [7]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [8]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [10]:
%%sql
SELECT title
FROM   films

 * sqlite:///movies.sqlite
Done.


title
Willy Wonka & the Chocolate Factory
Alien
WHAT DID JACK DO?
Dont Look Back


+ Show the performance dates for one of the movies.

In [11]:
%%sql
SELECT date
FROM screenings
JOIN films
USING (imdb_id)
WHERE title = 'Alien'

 * sqlite:///movies.sqlite
Done.


date
2022-02-26


+ Show all data concerning performances at a given theatere
  on a given date.

In [15]:
%%sql
SELECT *
FROM screenings
JOIN films
USING (imdb_id)
WHERE date = '2022-03-12' AND theatre = 'Kino'

 * sqlite:///movies.sqlite
Done.


theatre,date,time,imdb_id,title,year,run_time
Kino,2022-03-12,20:00,tt0067992,Willy Wonka & the Chocolate Factory,1971,100


+ List all customers

In [16]:
%%sql
SELECT *
FROM customers

 * sqlite:///movies.sqlite
Done.


username,first_name,last_name,password
nichobi,Nicholas,Boyd Isacsson,password
mollorg,Mollie,Slater,password
wildman01,Kyle,Wildman,password
erma32,Erik,Gullberg,password
bellsebub,Bella,Krantz,password


+ List all tickets

In [17]:
%%sql
SELECT *
from tickets

 * sqlite:///movies.sqlite
Done.


uuid,theatre,date,time,customer
d81bb73d1a78c57786d7e8c85cbf77f0,Kino,2022-03-12,20:00,nichobi
f15c8872a4271f1407a5ee3e670dfc86,Kino,2022-03-12,20:00,mollorg
10dd0c916c590fe6c61b91e1a28c21cf,Filmstaden,2022-02-26,21:15,erma32
b4e8149da9f5c6899cc9a9c55ee1d04a,Filmstaden,2022-02-26,21:15,wildman01
75ee297af6c234a64ffe56a16f34b6c6,Kino,2022-06-10,15:30,bellsebub


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [18]:
%%sql
INSERT
INTO    tickets (theatre, date, time, customer)
VALUES  ('Kino', '2022-06-10', '15:30', 'nichobi');

 * sqlite:///movies.sqlite
1 rows affected.


[]

In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [28]:
%%sql
--INSERT
--INTO    tickets (theatre, date, time, customer)
--VALUES  ('Kino', '2022-06-10', '15:30', 'nichobi')
--RETURNING uuid
SELECT uuid
FROM   tickets
WHERE  rowid = last_insert_rowid();


 * sqlite:///movies.sqlite
Done.


uuid


+ Try to insert two movie theaters with the same name (this
  should fail).

In [31]:
%%sql
INSERT
INTO    theatres (name, capacity)
VALUES  ('Kino', 100);

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: theatres.name
[SQL: INSERT INTO theatres (name, capacity)
VALUES  ('Kino', 100);]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [32]:
%%sql
INSERT
INTO    screenings (theatre, date, time, imdb_id)
VALUES  ('Kinno', '2022-03-12', '20:00', 'tt0067992');


 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO screenings (theatre, date, time, imdb_id)
VALUES  ('Kinno', '2022-03-12', '20:00', 'tt0067992');]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

+ Create a ticket where either the user or the performance
  doesn’t exist (this should fail).

In [35]:
%%sql
INSERT
INTO    tickets (theatre, date, time, customer)
--VALUES  ('Kino', '2022-03-12', '20:00', 'nicobi');
VALUES  ('Kino', '2022-03-12', '20:01', 'nichobi');

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO tickets (theatre, date, time, customer)
--VALUES  ('Kino', '2022-03-12', '20:00', 'nicobi');
VALUES  ('Kino', '2022-03-12', '20:01', 'nichobi');]
(Background on this error at: https://sqlalche.me/e/14/gkpj)